In [1]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from statsmodels.stats.diagnostic import acorr_breusch_godfrey

1. state alphabetical; DC = 9
2. year 80 to 93
5. metro proportion in metro. areas
6. unem proportion unemployed
27. ag15_17 prop. pop. 15 to 17 yrs
28. ag18_24 prop. pop. 18 to 24 yrs
30. incpc per capita income, nominal
31. polpc police per 100,000 residents
41. pris prison pop. per 100,000

In [4]:
# Carregar a base
df = pd.read_stata("Base2.dta")

# Definir o identificador do painel
df = df.set_index(["state", "year"])  # Supondo que "state" e "year" sejam os nomes das colunas

# Mostrar estatísticas descritivas das variáveis de interesse
print("Estatísticas descritivas das variáveis de interesse:")
print(df[["criv", "polpc", "incpc", "pris", "unem", "ag15_17", "ag18_24", "metro"]].describe())

# Criar termo defasado de polpc
df["polpc_lag"] = df.groupby(level=0)["polpc"].shift(1)

# Selecionar variáveis
X = df[["polpc", "incpc", "pris", "unem", "ag15_17", "ag18_24", "metro"]]
X = sm.add_constant(X)  # Adicionar constante
Y = df["criv"]

# Estimar o modelo com efeitos fixos para estado e ano
model = PanelOLS(Y, X, entity_effects=True, time_effects=True, drop_absorbed=True)
results = model.fit()

# Exibir resultados
print(results.summary)

# Método da primeira diferença
df_diff = df.groupby(level=0).diff().dropna()
X_diff = df_diff[["polpc", "incpc", "pris", "unem", "ag15_17", "ag18_24", "metro"]]
X_diff = sm.add_constant(X_diff)  # Adicionar constante
Y_diff = df_diff["criv"]

model_diff = sm.OLS(Y_diff, X_diff).fit()
print(model_diff.summary())

# Teste de autocorrelação serial (Breusch-Godfrey)
residuals = model_diff.resid
bg_test = acorr_breusch_godfrey(model_diff, nlags=1)  # Teste de 1 defasagem
print("Breusch-Godfrey Test:", bg_test)

# Interpretação do teste de Breusch-Godfrey
p_value = bg_test[1]
if p_value < 0.05:
    print("Há evidências de autocorrelação serial nos resíduos (p-valor < 0.05).")
else:
    print("Não há evidências de autocorrelação serial nos resíduos (p-valor >= 0.05).")

Estatísticas descritivas das variáveis de interesse:
             criv       polpc         incpc         pris        unem  \
count  714.000000  714.000000    714.000000   714.000000  714.000000   
mean     5.078585  267.807983  14781.809570   199.980347    0.068487   
std      3.440077   86.722107   4086.768311   132.713470    0.022151   
min      0.475628  161.118881   6883.166992    20.826952    0.021420   
25%      2.824363  220.732796  11605.039795   113.038683    0.053000   
50%      4.515549  249.902298  14263.350586   172.454063    0.066000   
75%      6.500603  288.558533  17541.873047   251.896126    0.080958   
max     29.217993  907.947083  29859.474609  1286.837524    0.180330   

          ag15_17     ag18_24       metro  
count  714.000000  714.000000  714.000000  
mean     0.045864    0.116925    0.641728  
std      0.005494    0.013811    0.225690  
min      0.025952    0.084802    0.153000  
25%      0.042143    0.105736    0.457850  
50%      0.046027    0.116627    0